In [ ]:
import pandas as pa
from GetFreshMarketData import *

In [ ]:
end_date = datetime.today()
start_date = end_date - timedelta(days = 365 * 3)
past_six_months = end_date - timedelta(days = 90)
file_cr = 20000000

all_traded_stock = pd.read_csv(INDEX_DIR_CONST/'all_traded_stock.csv',low_memory=False)
all_traded_stock.date = all_traded_stock.date.apply(lambda x : datetime.strptime(x,'%Y-%m-%d'))
all_traded_stock = all_traded_stock.loc[((all_traded_stock.date <= end_date) & 
                                        (all_traded_stock.date >= past_six_months) &
                                        (all_traded_stock.total_value >=file_cr)),:]
all_traded_stock = all_traded_stock.groupby('symbol').agg({'date':len}).reset_index()
candidate_symbol = all_traded_stock.loc[all_traded_stock.date==all_traded_stock.date.max(),'symbol']

In [101]:
progress_bar = tqdm(INDEX_TICKERS_DF.iterrows(), desc="Downloading historical data", total=len(INDEX_TICKERS_DF.NSE_SYMBOL))

relative_performace_df = pd.DataFrame(columns =['index_name','roc'])
index_contituents_df = pd.DataFrame(columns = ['symbol','index_name'])
all_index_dict = {}
for _, row in progress_bar:
    symbol = row['NSE_SYMBOL']
    fl_name = row['INDEX_NAME']
    index  = pd.read_csv(STOCK_DIR/f'{symbol}.csv')
    index_constituents = pd.read_csv(INDEX_DIR_CONST/f'{fl_name}.csv')
    index.date = pd.to_datetime(index.date)
    index = index.set_index('date')
    
    index_constituents['index_name'] = symbol
    index_contituents_df = pd.concat([index_contituents_df,index_constituents[['symbol','index_name']]])
    
    weekly_index_close = index.resample('W-MON').agg({'close': 'last'})
    weekly_index_close = weekly_index_close.ffill()
    

    if relative_performace_df.empty:
        relative_performace_df = pd.DataFrame({'index_name':[symbol],
                                                        'roc':weekly_index_close.pct_change(1).tail(1).values[0]})
    else:
        relative_performace_df = pd.concat([relative_performace_df,pd.DataFrame({'index_name':[symbol],
                                                        'roc':weekly_index_close.pct_change(1).tail(1).values[0]})])

    all_index_dict[symbol] = index


relative_performace_df = relative_performace_df.sort_values(by='roc')

In [106]:
def calculate_bollinger_bands(close: pd.Series, period: int = 50, sigma: int = 2) -> pd.DataFrame:
                """Calculate Bollinger Bands"""
                rolling_mean = close.rolling(window=period).mean()
                rolling_std = close.rolling(window=period).std()
                upper_band = rolling_mean + (rolling_std * sigma)
                lower_band = rolling_mean - (rolling_std * sigma)
                return pd.DataFrame({
                    'upper_band': upper_band,
                    'lower_band': lower_band
                }, index=close.index)

In [116]:
pbar = tqdm(candidate_symbol,desc = "Processing for ",total=len(candidate_symbol))
final_stocks = pd.DataFrame()
for symbol in pbar:
    file = STOCK_DIR/f'{symbol}.csv'
    if file.exists():
        df = pd.read_csv(file, low_memory=False)
        df.date = df.date.apply(lambda x : datetime.strptime(x,'%Y-%m-%d'))
        df = df.set_index('date')
        df = df.sort_index()

        weekly_index_close = df.resample('W-MON').agg({'close': 'last'})
        weekly_index_close = weekly_index_close.ffill()
        if weekly_index_close.pct_change(1).tail(1).values[0] >= relative_performace_df.roc.iloc[-1] :
            bb = calculate_bollinger_bands(df['close'])
            latest_close = df['close'].iloc[-1]
            latest_upper = bb['upper_band'].iloc[-1]
            if pd.notna(latest_upper) and latest_close > latest_upper:
                temp = pd.DataFrame({'symbol': symbol,
                                    'weekly_return': weekly_index_close.pct_change(1).tail(1).values[0],
                                    'latest_close': latest_close,
                                    'upper_bb': latest_upper,
                                    'bb_breakout': latest_close / latest_upper - 1
                                    })
                final_stocks = pd.concat([final_stocks, temp])
final_stocks = final_stocks.sort_values(by='weekly_return')

Processing for :   0%|          | 0/975 [00:00<?, ?it/s]

Processing for : 100%|██████████| 975/975 [00:48<00:00, 20.26it/s]


In [117]:
final_stocks

,symbol,weekly_return,latest_close,upper_bb,bb_breakout
0,WIPRO,0.010354,264.45,263.994319,0.001726
0,GODREJCP,0.017237,1186.20,1178.490142,0.006542
0,INFY,0.019853,1638.70,1637.100839,0.000977
0,HINDCOPPER,0.020003,387.55,381.732154,0.015241
0,HDBFS,0.021478,772.85,767.777184,0.006607
0,PHOENIXLTD,0.023416,1831.30,1816.252704,0.008285
0,TECHM,0.023550,1612.50,1610.559314,0.001205
0,SUNDARMFIN,0.027267,4931.60,4926.448126,0.001046
0,DIAMONDYD,0.028186,1134.50,1121.768381,0.011350
0,HEXT,0.030356,792.55,785.367160,0.009146


TypeError: '<=' not supported between instances of 'str' and 'datetime.datetime'

In [82]:
all_traded_stock.total_value[0]

np.float64(10827794.9)